In [313]:
import re
import math
import random
import tkinter as tk
from tkinter import filedialog, simpledialog, Frame, Label, Entry, LEFT, RIGHT, END

In [314]:
class Node:
    radius = 10
    small_radius = 3
    
    def __init__(self, network, x, y, text):
        self.index = -1
        self.network = network
        self.x = x
        self.y = y
        self.text = text
        self.links = []
        self.is_start_node = False
        self.is_end_node = False
        self.from_node = None
        
        self.network.add_node(self)
        
        self.visited = False
        self.distance = float('inf')
        self.shortest_path_link = None
        
    def __str__(self):
        return f"[{self.text}]"
    
    def add_link(self, link):
        self.links.append(link)
        
    def draw(self, canvas, draw_labels=True):
        radius = self.radius if draw_labels else self.small_radius
        oval_id = canvas.create_oval(
            self.x - radius, self.y - radius,
            self.x + radius, self.y + radius,
            fill='pink' if self.is_start_node else 'lightblue1' if self.is_end_node else 'white',
            outline='black'
        )
        canvas.tag_bind(oval_id, '<Button-1>', lambda evt: self.network.select_start_node(self))
        canvas.tag_bind(oval_id, '<Button-3>', lambda evt: self.network.select_end_node(self))
        if draw_labels:
            text_id = canvas.create_text(
                self.x, self.y,
                text=self.text,
                fill='blue'
            )
            canvas.tag_bind(text_id, '<Button-1>', lambda evt: self.network.select_start_node(self))
            canvas.tag_bind(text_id, '<Button-3>', lambda evt: self.network.select_end_node(self))

class Link:
    def __init__(self, network, from_node, to_node, cost):
        self.network = network
        self.from_node = from_node
        self.to_node = to_node
        self.cost = cost
        self.is_in_tree = False
        self.is_in_path = False
    
        self.network.add_link(self)
        self.from_node.add_link(self)
        
    def __str__(self):
        return f"[{self.from_node.text}] --> [{self.to_node.text}] ({self.cost})"
    
    def draw(self, canvas):
        # Draw link one way only, avoid having to draw bidirectional links twice
        if not self.is_in_tree and not self.is_in_path and self.from_node.index > self.to_node.index:
            return

        canvas.create_line(
            self.from_node.x,
            self.from_node.y,
            self.to_node.x,
            self.to_node.y,
            fill='red' if self.is_in_path else 'green' if self.is_in_tree else 'black',
            width=5 if self.is_in_tree or self.is_in_path else 1
        )
    
    def draw_label(self, canvas):
        # Draw a label along this link.
        dx = self.to_node.x - self.from_node.x
        dy = self.to_node.y - self.from_node.y
        
        angle_radians = math.atan2(dx, dy)
        angle_degrees = angle_radians * 180 / math.pi
        # Subtract 90 degrees so that text will run parallel to the link
        angle_degrees -= 90
        
        # Find the point one-third of the way along link, using weighted average
        x = 0.67 * self.from_node.x + 0.33 * self.to_node.x
        y = 0.67 * self.from_node.y + 0.33 * self.to_node.y
        
        # Erase part of the link
        radius = 10
        canvas.create_oval(
            x - radius, y - radius,
            x + radius, y + radius,
            fill='white',
            outline=''
        )
        
        canvas.create_text(
            x, y,
            text=str(self.cost),
            angle=angle_degrees
        )

class Network:
    def __init__(self, canvas=None, status_var=None):
        self.canvas = canvas
        self.status_var = status_var
        self.clear()
    
    def clear(self):
        self.nodes = []
        self.links = []
        self.start_node = None
        self.end_node = None
        
    def add_node(self, node: Node):
        node.index = len(self.nodes)
        self.nodes.append(node)
        
    def add_link(self, link: Link):
        self.links.append(link)
        
    def to_string(self):
        return (
            "{} # Num nodes.".format(len(self.nodes)) +
            "\n{} # Num links.".format(len(self.links)) +
            "\n# Nodes." +
            "\n{}".format('\n'.join([f"{n.x},{n.y},{n.text}" for n in self.nodes])) +
            "\n# Links." +
            "\n{}".format('\n'.join([f"{l.from_node.index},{l.to_node.index},{l.cost}" for l in self.links]))
        )
    
    def save_into_file(self, file_name):
        open(file_name, 'w').write(self.to_string())
    
    @staticmethod
    def read_next_line(file_handle):
        while line := file_handle.readline():
            pattern = re.compile('([^#]*)(#.*)?$')
            matcher = pattern.match(line)
            line = matcher.group(1).strip() if matcher else None
            if line:
                return line 
        return None
    
    def load_from_file(self, file_name):
        self.clear()
        
        with open(file_name, 'r') as file_handle:
            num_nodes = int(self.read_next_line(file_handle))
            num_links = int(self.read_next_line(file_handle))
            # get nodes
            while num_nodes > 0 and (line := self.read_next_line(file_handle)):
                num_nodes -= 1
                if m := re.match(r'(\d+),(\d+),(.*)', line):
                    x = int(m.group(1))
                    y = int(m.group(2))
                    text = m.group(3)
                    Node(self, x, y, text)
            # get links
            while num_links > 0 and (line := self.read_next_line(file_handle)):
                num_links -= 1
                if m := re.match(r'(\d+),(\d+),(\d+)', line):
                    from_node_idx = int(m.group(1))
                    to_node_idx = int(m.group(2))
                    cost = int(m.group(3))
                    Link(self, self.nodes[from_node_idx], self.nodes[to_node_idx], cost)

    def draw(self):
        if not self.canvas: pass
        
        # Clear any previous drawing.
        self.canvas.delete('all')
        
        draw_labels = True if len(self.nodes) < 100 else False
        
        for l in self.links:
            l.draw(self.canvas)
        if draw_labels:
            for l in self.links:
                l.draw_label(self.canvas)
            
        for n in self.nodes:
            n.draw(self.canvas, draw_labels)
    
    def select_start_node(self, node):
        if self.start_node:
            self.start_node.is_start_node = False
        self.start_node = node
        self.start_node.is_start_node = True
        self.check_for_path()
        self.draw()
    
    def select_end_node(self, node):
        if self.end_node:
            self.end_node.is_end_node = False
        self.end_node = node
        self.end_node.is_end_node = True
        self.check_for_path()
        self.draw()
        
    def check_for_path(self):
        if self.start_node:
            self.find_path_tree_label_setting()
        
            if self.end_node:
                self.find_path()
    
    def find_path_tree_label_setting(self):
        # clear all previous labelled nodes and paths
        for n in self.nodes:
            n.visited = False
            n.distance = float('inf')
            n.shortest_path_link = None
            for l in n.links:
                l.is_in_tree = False
                l.is_in_path = False
                
        # initialise stats
        num_checks = 0
        num_pops = 0
        
        # initialise starting distance
        self.start_node.distance = 0
        # place current node to the candidate list
        candidate_list = [self.start_node]
        
        while len(candidate_list) > 0:
            best_distance = float('inf')
            best_node = None
            for n in list(candidate_list):
                num_checks += 1
                if best_distance > n.distance:
                    best_distance = n.distance
                    best_node = n
            
            best_node.visited = True
            num_pops += 1
            candidate_list.remove(best_node)
            
            # consider neighbouring nodes
            for l in best_node.links:
                if l.to_node.visited:
                   continue
                
                # improve neighbouring node's distance
                new_best_distance = best_distance + l.cost
                if new_best_distance < l.to_node.distance:
                    l.to_node.distance = new_best_distance
                    l.to_node.shortest_path_link = l
                    
                    # mark as the shortest paths to node
                    l.is_in_tree = True
                    
                    candidate_list.append(l.to_node)
        
        # display stats
        if self.status_var:
            self.status_var.set(f"Checks: [{'{0:,}'.format(num_checks)}] - Pops: [{'{0:,}'.format(num_pops)}]")
    
    def find_path(self):
        n = self.end_node
        while n.shortest_path_link:
            if n.shortest_path_link:
                n.shortest_path_link.is_in_path = True
                n = n.shortest_path_link.from_node
        # display distance to end node
        if self.status_var:
            existing_stats = self.status_var.get()
            self.status_var.set(f"{existing_stats} - Total cost: [{'{0:,}'.format(self.end_node.distance)}]")


In [315]:
def create_test_network_popup(toplevel):
    from datetime import datetime
    entry_size = 16
    
    width_frame = Frame(toplevel)
    width_frame.pack(padx=2, pady=2)
    
    width_label = Label(width_frame, text='Width', justify='right')
    width_label.pack(padx=2, side=LEFT)
    toplevel.width = tk.IntVar()
    toplevel.width.set(600)
    width = Entry(width_frame, textvariable=toplevel.width, width=entry_size)
    width.pack(padx=2, side=RIGHT)
    
    height_frame = Frame(toplevel)
    height_frame.pack(padx=2, pady=2)
    
    height_label = Label(height_frame, text='Height', justify='right')
    height_label.pack(padx=2, side=LEFT)
    toplevel.height = tk.IntVar()
    toplevel.height.set(400)
    height = Entry(height_frame, textvariable=toplevel.height, width=entry_size)
    height.pack(padx=2, side=RIGHT)

    rows_frame = Frame(toplevel)
    rows_frame.pack(padx=2, pady=2)
    
    rows_label = Label(rows_frame, text='Rows', justify='right')
    rows_label.pack(padx=2, side=LEFT)
    toplevel.rows = tk.IntVar()
    toplevel.rows.set(6)
    rows = Entry(rows_frame, textvariable=toplevel.rows, width=entry_size)
    rows.pack(padx=2, side=RIGHT)

    cols_frame = Frame(toplevel)
    cols_frame.pack(padx=2, pady=2)
    
    cols_label = Label(cols_frame, text='Columns', justify='right')
    cols_label.pack(padx=2, side=LEFT)
    toplevel.cols = tk.IntVar()
    toplevel.cols.set(10)
    cols = Entry(cols_frame, textvariable=toplevel.cols, width=entry_size)
    cols.pack(padx=2, side=RIGHT)

    filename_frame = Frame(toplevel)
    filename_frame.pack(padx=2, pady=2)
    
    filename_label = Label(filename_frame, text='File name', justify='right')
    filename_label.pack(padx=2, side=LEFT)
    toplevel.filename = Entry(filename_frame, width=entry_size*2)
    toplevel.filename.insert(END, f"network-{datetime.now().strftime('%Y%m%d%H%M%S')}.net")
    toplevel.filename.pack(padx=2, side=RIGHT)
    # return widget that should have the focus
    return width


In [316]:
class App:
    # Create and manage the tkinter interface.
    def __init__(self):
        self.network = None
        
        # Make the main interface.
        self.window = tk.Tk()
        self.window.title('label_setting')
        self.window.protocol('WM_DELETE_WINDOW', self.kill_callback)
        self.window.geometry('600x400')
        
        # Build the menu.
        self.menubar = tk.Menu(self.window)
        self.menu_file = tk.Menu(self.menubar, tearoff=False)
        self.menu_file.add_command(label='Open...', command=self.open_network,
                                   accelerator='Ctrl+O')
        self.menu_file.add_command(label='Create Network...', command=self.create_test_network,
                                   accelerator='Ctrl-N')
        self.menu_file.add_separator()
        self.menu_file.add_command(label='Exit', command=self.kill_callback)
        self.menubar.add_cascade(label='File', menu=self.menu_file)
        self.window.config(menu=self.menubar)
        
        # Build the window surface.
        self.canvas = tk.Canvas(self.window, bg='white', borderwidth=2, relief=tk.SUNKEN)
        self.canvas.pack(padx=10, pady=(0, 5), fill=tk.BOTH, expand=True)
        self.window.bind('<Control-o>', self.ctrl_o_pressed)
        self.window.bind('<Control-n>', self.ctrl_n_pressed)
        
        # Status bar
        self.status_var = tk.StringVar(value='')
        status = Label(self.window, textvariable=self.status_var, bd=1, relief=tk.FLAT, anchor=tk.E)
        status.pack(padx=10, pady=(0, 10), side=tk.BOTTOM, fill=tk.BOTH)
        
        # Display the window.
        self.window.focus_force()
        self.window.mainloop()
        
    def kill_callback(self):
        self.window.destroy()
        
    def ctrl_o_pressed(self, event):
        self.open_network()
    def open_network(self):
        if file_name := filedialog.askopenfilename(initialdir='./'):
            self.canvas.delete("all")
            self.network = Network(self.canvas, self.status_var)
            self.network.load_from_file(file_name)
            self.draw_network()
    
    def ctrl_n_pressed(self, event):
        self.create_test_network()
    @staticmethod
    def create_test_network_ok_pressed(toplevel):
        build_grid_network(
            toplevel.filename.get(), 
            toplevel.width.get(), 
            toplevel.height.get(),
            toplevel.rows.get(), 
            toplevel.cols.get()
        )
        toplevel.destroy()
    def create_test_network(self):
        (type(
            '',                     # anonymous, leave class name blank
            (simpledialog.Dialog,), # base classes
            {                       # body
                "body": lambda toplevel, master: create_test_network_popup(toplevel),
                "ok": lambda toplevel: self.create_test_network_ok_pressed(toplevel)
            }
        )(self.window, 'Create Test Network'))
    
    def draw_network(self):
        if self.network:
            self.network.draw()


In [317]:
App()


In [318]:
def build_grid_network(file_name, width, height, num_rows, num_cols):
    network = Network()
    node_id = 0
    margin = 10
    x = 0 + margin + Node.radius
    y = 0 + margin + Node.radius

    x_space = int((width - 2 * margin) / num_cols)
    y_space = int((height - 2 * margin) / num_rows)

    # Nodes will be stored in the network's instance in a list
    for r in range(0, num_rows):
        for c in range(0, num_cols):
            Node(network, x, y, node_id)
            node_id += 1
            x += x_space + Node.radius
            
        x = 0 + margin + Node.radius
        y += y_space + Node.radius

    # When creating links refer to network's nodes list
    for r in range(0, num_rows):
        for c in range(0, num_cols):
            node = network.nodes[r*num_cols + c]
            
            adjacent_top = (r-1)*num_cols + c if r > 0 else None
            if adjacent_top is not None:
                to_node = network.nodes[adjacent_top]
                Link(
                    network,
                    node,
                    to_node,
                    math.dist([node.x, node.y], [to_node.x, to_node.y]) * random.uniform(1.0, 1.2)
                )
            adjacent_right = r*num_cols + c+1 if c < num_cols-1 else None
            if adjacent_right is not None:
                to_node = network.nodes[adjacent_right]
                Link(
                    network,
                    node,
                    to_node,
                    math.dist([node.x, node.y], [to_node.x, to_node.y]) * random.uniform(1.0, 1.2)
                )
            adjacent_bottom = (r+1)*num_cols + c if r < num_rows-1 else None
            if adjacent_bottom is not None:
                to_node = network.nodes[adjacent_bottom]
                Link(
                    network,
                    node,
                    to_node,
                    math.dist([node.x, node.y], [to_node.x, to_node.y]) * random.uniform(1.0, 1.2)
                )
            adjacent_left = r*num_cols + c-1 if c > 0 else None
            if adjacent_left is not None:
                to_node = network.nodes[adjacent_left]
                Link(
                    network,
                    node,
                    to_node,
                    math.dist([node.x, node.y], [to_node.x, to_node.y]) * random.uniform(1.0, 1.2)
                )
    
    network.save_into_file(file_name)
